# 3. Train-Predict LSTM

## Result:
   - Kaggle score:
    
## Tensorboard:
   - Input at command: tensorboard --logdir=./log
   - Input at browser: http://127.0.0.1:6006
    
## Reference:
   - https://www.kaggle.com/codename007/a-very-extensive-landmark-exploratory-analysis

## Import PKGs

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
from IPython.display import display

import os
import gc
import time
import zipfile
import pickle
from PIL import Image
import shutil

from tqdm import tqdm
import multiprocessing

## Run name

In [2]:
project_name = 'ic_furniture2018'
step_name = 'Train-Predict_LSTM'
time_str = time.strftime("%Y%m%d_%H%M%S", time.localtime())
run_name = project_name + '_' + step_name + '_' + time_str
print('run_name: ' + run_name)

run_name: ic_furniture2018_Train-Predict_LSTM_20180330_140815


## Project folders

In [3]:
cwd = os.getcwd()
input_folder = os.path.join(cwd, 'input')
output_folder = os.path.join(cwd, 'output')
model_folder = os.path.join(cwd, 'model')
feature_folder = os.path.join(cwd, 'feature')
post_pca_feature_folder = os.path.join(cwd, 'post_pca_feature')
log_folder = os.path.join(cwd, 'log')
print('input_folder: \t\t\t%s' % input_folder)
print('output_folder: \t\t\t%s' % output_folder)
print('model_folder: \t\t\t%s' % model_folder)
print('feature_folder: \t\t%s' % feature_folder)
print('post_pca_feature_folder: \t%s' % post_pca_feature_folder)
print('log_folder: \t\t\t%s' % log_folder)

org_train_folder = os.path.join(input_folder, 'org_train')
org_validation_folder = os.path.join(input_folder, 'org_validation')
org_test_folder = os.path.join(input_folder, 'org_test')
train_folder = os.path.join(input_folder, 'data_train')
validation_folder = os.path.join(input_folder, 'data_validation')
test_folder = os.path.join(input_folder, 'data_test')
test_sub_folder = os.path.join(test_folder, 'test')

if not os.path.exists(post_pca_feature_folder):
    os.mkdir(post_pca_feature_folder)
    print('Create folder: %s' % post_pca_feature_folder)

train_json_file = os.path.join(input_folder, 'train.json')
validation_json_file = os.path.join(input_folder, 'validation.json')
test_json_file = os.path.join(input_folder, 'test.json')
print('\ntrain_json_file: \t\t%s' % train_json_file)
print('validation_json_file: \t\t%s' % validation_json_file)
print('test_json_file: \t\t%s' % test_json_file)

train_csv_file = os.path.join(input_folder, 'train.csv')
validation_csv_file = os.path.join(input_folder, 'validation.csv')
test_csv_file = os.path.join(input_folder, 'test.csv')
print('\ntrain_csv_file: \t\t%s' % train_csv_file)
print('validation_csv_file: \t\t%s' % validation_csv_file)
print('test_csv_file: \t\t\t%s' % test_csv_file)

sample_submission_csv_file = os.path.join(input_folder, 'sample_submission_randomlabel.csv')
print('\nsample_submission_csv_file: \t%s' % sample_submission_csv_file)

input_folder: 			/data1/kaggle/imaterialist-challenge-furniture-2018/input
output_folder: 			/data1/kaggle/imaterialist-challenge-furniture-2018/output
model_folder: 			/data1/kaggle/imaterialist-challenge-furniture-2018/model
feature_folder: 		/data1/kaggle/imaterialist-challenge-furniture-2018/feature
post_pca_feature_folder: 	/data1/kaggle/imaterialist-challenge-furniture-2018/post_pca_feature
log_folder: 			/data1/kaggle/imaterialist-challenge-furniture-2018/log

train_json_file: 		/data1/kaggle/imaterialist-challenge-furniture-2018/input/train.json
validation_json_file: 		/data1/kaggle/imaterialist-challenge-furniture-2018/input/validation.json
test_json_file: 		/data1/kaggle/imaterialist-challenge-furniture-2018/input/test.json

train_csv_file: 		/data1/kaggle/imaterialist-challenge-furniture-2018/input/train.csv
validation_csv_file: 		/data1/kaggle/imaterialist-challenge-furniture-2018/input/validation.csv
test_csv_file: 			/data1/kaggle/imaterialist-challenge-furniture-2018/inp

## Preview data

In [4]:
train_csv = pd.read_csv(train_csv_file)
print('train_csv.shape is {0}.'.format(train_csv.shape))
display(train_csv.head(2))

validation_csv = pd.read_csv(validation_csv_file)
print('train_csv.shape is {0}.'.format(validation_csv.shape))
display(validation_csv.head(2))

test_csv = pd.read_csv(test_csv_file)
print('test_csv.shape is {0}.'.format(test_csv.shape))
display(test_csv.head(2))

test_csv = pd.read_csv(test_csv_file)
print('test_csv.shape is {0}.'.format(test_csv.shape))
display(test_csv.head(2))

sample_submission_csv = pd.read_csv(sample_submission_csv_file)
print('sample_submission_csv.shape is {0}.'.format(sample_submission_csv.shape))
display(sample_submission_csv.head(2))

train_csv.shape is (194828, 3).


,image_id,label_id,url
0,1,5,https://img13.360buyimg.com/imgzone/jfs/t2857/...
1,2,5,http://www.tengdakeli.cn/350/timg01/uploaded/i...


train_csv.shape is (6400, 3).


,image_id,label_id,url
0,1,38,http://www.ghs.net/public/images/fb/3d/51/3beb...
1,2,63,https://img.alicdn.com/imgextra/TB2chFei9YH8KJ...


test_csv.shape is (12800, 2).


,image_id,url
0,1,https://img13.360buyimg.com/imgzone/jfs/t13174...
1,2,http://img35.ddimg.cn/79/22/1258168705-1_u.jpg


test_csv.shape is (12800, 2).


,image_id,url
0,1,https://img13.360buyimg.com/imgzone/jfs/t13174...
1,2,http://img35.ddimg.cn/79/22/1258168705-1_u.jpg


sample_submission_csv.shape is (12800, 2).


,id,predicted
0,1,57
1,2,74


In [5]:
train_id = train_csv['image_id']
train_label_id = train_csv['label_id']

id_2_train_label_id_dict = dict(zip(train_id, train_label_id))
print('len(id_2_train_label_id_dict)=%d' % len(id_2_train_label_id_dict))

index = 0
print('id: %s, \tlandmark_id:%s' % (train_id[index], id_2_train_label_id_dict[train_id[index]]))
index = 1
print('id: %s, \tlandmark_id:%s' % (train_id[index], id_2_train_label_id_dict[train_id[index]]))

image_file = '%s_%s.jpg' % (train_id[index], id_2_train_label_id_dict[train_id[index]])
print(image_file)

len(id_2_train_label_id_dict)=194828
id: 1, 	landmark_id:5
id: 2, 	landmark_id:5
2_5.jpg


In [6]:
val_id = val_csv['image_id']
val_label_id = val_csv['label_id']

id_2_val_label_id_dict = dict(zip(val_id, val_label_id))
print('len(id_2_val_label_id_dict)=%d' % len(id_2_val_label_id_dict))

index = 0
print('id: %s, \tlandmark_id:%s' % (val_id[index], id_2_val_label_id_dict[val_id[index]]))
index = 1
print('id: %s, \tlandmark_id:%s' % (val_id[index], id_2_val_label_id_dict[val_id[index]]))

image_file = '%s_%s.jpg' % (val_id[index], id_2_val_label_id_dict[val_id[index]])
print(image_file)

len(id_2_validation_label_id_dict)=6400
id: 1, 	landmark_id:38
id: 2, 	landmark_id:63
2_63.jpg


In [7]:
test_id = test_csv['image_id']

index = 0
print('id: %s' % (test_id[index]))
index = 1
print('id: %s' % (test_id[index]))

image_file = '%s.jpg' % (test_id[index])
print(image_file)

id: 1
id: 2
2.jpg


## Load feature

In [8]:
%%time
import h5py
import numpy as np
from sklearn.utils import shuffle
np.random.seed(2018)


def load_h5(model_name, time_str, post_pca_feature_folder):
    x_train = {}
    y_train = {}
    x_val = {}
    y_val = {}
    x_test = {}
    
    feature_model = os.path.join(post_pca_feature_folder, 'feature_%s_%s.h5' % (model_name, time_str))
    for filename in [feature_model]:
        with h5py.File(filename, 'r') as h:
            x_train = np.array(h['train'])
            y_train = np.array(h['train_labels'])
            x_val = np.array(h['val'])
            y_val = np.array(h['val_labels'])
            x_test = np.array(h['test'])
    return x_train, y_train, x_val, y_val, x_test

CPU times: user 116 ms, sys: 12 ms, total: 128 ms
Wall time: 124 ms


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [9]:
%%time
x_train_Xception, y_train_Xception, x_val_Xception, y_val_Xception, x_test_Xception = load_h5('Xception', '20180329-164850', feature_folder)
print('*' * 60)
print(x_train_Xception.shape)
print(len(y_train_Xception))
print(x_val_Xception.shape)
print(len(y_val_Xception))
print(x_test_Xception.shape)

x_train_InceptionV3, y_train_InceptionV3, x_val_InceptionV3, y_val_InceptionV3, x_test_InceptionV3 = load_h5('InceptionV3', '20180329-164850', feature_folder)
print('*' * 60)
print(x_train_InceptionV3.shape)
print(len(y_train_InceptionV3))
print(x_val_InceptionV3.shape)
print(len(y_val_InceptionV3))
print(x_test_InceptionV3.shape)

************************************************************
(191261, 2048)
191261
(6301, 2048)
6301
(12652, 2048)
************************************************************
(191261, 2048)
191261
(6301, 2048)
6301
(12652, 2048)
CPU times: user 796 ms, sys: 3.91 s, total: 4.7 s
Wall time: 4.69 s


In [10]:
import sys
def describe(arr):
    print(arr.shape, arr.min(), arr.max(), sys.getsizeof(arr))

In [11]:
%%time
x_train = np.concatenate([x_train_Xception[:, np.newaxis, :],
                         x_train_InceptionV3[:, np.newaxis, :]], axis=1)
describe(x_train)

x_val = np.concatenate([x_val_Xception[:, np.newaxis, :],
                         x_val_InceptionV3[:, np.newaxis, :]], axis=1)
describe(x_val)
# del band1_data
# del band2_data
# del band_avg_data
gc.collect()

x_test = np.concatenate([x_test_Xception[:, np.newaxis, :],
                         x_test_InceptionV3[:, np.newaxis, :]], axis=1)
describe(x_test)
# del band1_test
# del band2_test
# del band_avg_test
gc.collect()

(191261, 2, 2048) 0.0 18.190805 3133620352
(6301, 2, 2048) 0.0 7.646922 103235712
(12652, 2, 2048) 0.0 10.15591 207290496
CPU times: user 1.4 s, sys: 1.34 s, total: 2.75 s
Wall time: 2.75 s


In [12]:
y_train = y_train_Xception
describe(y_train)
y_val = y_val_Xception
describe(y_val)

(191261,) 0 127 765140
(6301,) 0 127 25300


In [13]:
%%time
from sklearn.model_selection import train_test_split
# x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.01, random_state=5)

print(x_train.shape)
print(x_val.shape)

print(y_train.shape)
print(y_val.shape)

(191261, 2, 2048)
(6301, 2, 2048)
(191261,)
(6301,)
CPU times: user 136 ms, sys: 4 ms, total: 140 ms
Wall time: 140 ms


In [14]:
from keras.utils.np_utils import to_categorical

def data_generator(x_train, y_train, batch_size, num_classes):
    len_x_train = len(x_train)
    start_index = 0
    end_index = 0
    while(1):
        end_index = start_index + batch_size
        if end_index < len_x_train:
#             print(start_index, end_index, end=' ')
            x_batch = x_train[start_index: end_index, :]
            y_batch = y_train[start_index: end_index]
            y_batch_cat = to_categorical(y_batch, num_classes)
            
            start_index = start_index + batch_size
#             print(x_batch.shape, y_batch_cat.shape)
            yield x_batch, y_batch_cat
        else:
            end_index = end_index-len_x_train
#             print(start_index, end_index, end=' ')
            x_batch = np.vstack((x_train[start_index:, :], x_train[:end_index, :]))
            y_batch = np.concatenate([y_train[start_index:], y_train[:end_index]])
            y_batch_cat = to_categorical(y_batch, num_classes)
            
            start_index = end_index
#             print(x_batch.shape, y_batch_cat.shape)
            yield x_batch, y_batch_cat

Using TensorFlow backend.


In [15]:
data_dim = x_train.shape[2]
timesteps = x_train.shape[1]
num_classes = len(list(set(y_train)))
print(data_dim, timesteps, num_classes)
print('data_dim: \t%s' % data_dim)
print('timesteps: \t%s' % timesteps)
print('num_classes: \t%s' % num_classes)


print('*' * 60)
len_train = len(y_train)
batch_size = 128
steps_per_epoch_train = int(len_train/batch_size)
print('train data: %s %s %s' % (len_train, batch_size, steps_per_epoch_train))

train_gen = data_generator(x_train, y_train, batch_size, num_classes)
batch_data = next(train_gen)
print(batch_data[0].shape, batch_data[1].shape)


print('*' * 60)
len_val = len(y_val)
steps_per_epoch_val = int(len_val/batch_size)
print('val data: %s %s %s' % (len_val, batch_size, steps_per_epoch_val))

val_gen = data_generator(x_val, y_val, batch_size, num_classes)
batch_data = next(val_gen)
print(batch_data[0].shape, batch_data[1].shape)

2048 2 128
data_dim: 	2048
timesteps: 	2
num_classes: 	128
************************************************************
train data: 191261 128 1494
(128, 2, 2048) (128, 128)
************************************************************
val data: 6301 128 49
(128, 2, 2048) (128, 128)


## Build NN

In [16]:
from sklearn.metrics import confusion_matrix

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input, Flatten, Conv2D, MaxPooling2D, BatchNormalization, LSTM
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler, TensorBoard

In [17]:
def get_lr(x):
    lr = round(3e-4 * 0.98 ** x, 6)
    if lr < 1e-5:
        lr = 1e-5
    print(lr, end='  ')
    return lr

# annealer = LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)
annealer = LearningRateScheduler(get_lr)

log_dir = os.path.join(log_folder, run_name)
print('log_dir:' + log_dir)
tensorBoard = TensorBoard(log_dir=log_dir)

log_dir:/data1/kaggle/imaterialist-challenge-furniture-2018/log/ic_furniture2018_Train-Predict_LSTM_20180330_140815


In [18]:
# expected input data shape: (batch_size, timesteps, data_dim)
model = Sequential()
model.add(LSTM(256, return_sequences=True, input_shape=(timesteps, data_dim)))  # returns a sequence of vectors of dimension 32
model.add(LSTM(256, return_sequences=True))  # returns a sequence of vectors of dimension 32
model.add(LSTM(256))  # return a single vector of dimension 32
model.add(Dense(num_classes, activation='softmax'))

# model.compile(loss='categorical_crossentropy',
#               optimizer='rmsprop',
#               metrics=['accuracy'])
model.compile(optimizer=Adam(lr=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [19]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 2, 256)            2360320   
_________________________________________________________________
lstm_2 (LSTM)                (None, 2, 256)            525312    
_________________________________________________________________
lstm_3 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
Total params: 3,443,840
Trainable params: 3,443,840
Non-trainable params: 0
_________________________________________________________________


In [20]:
%%time
hist = model.fit_generator(train_gen,
    steps_per_epoch=steps_per_epoch_train,
    epochs=10, #Increase this when not on Kaggle kernel
    verbose=1,  #1 for ETA, 0 for silent
#     callbacks=[annealer],
    max_queue_size=2,
    workers=4,
    use_multiprocessing=False,
    validation_data=val_gen,
    validation_steps=steps_per_epoch_val)

Epoch 1/10
 394/1494 [======>.......................] - ETA: 29s - loss: 4.6976 - acc: 0.0490

KeyboardInterrupt: 

In [21]:
final_loss, final_acc = model.evaluate_generator(val_gen, steps=steps_per_epoch_val)
print("Final loss: {0:.4f}, final accuracy: {1:.4f}".format(final_loss, final_acc))

Final loss: 4.9690, final accuracy: 0.0080


In [22]:
run_name_acc = run_name + '_' + str(int(final_loss*10000)).zfill(4)

In [23]:
histories = pd.DataFrame(hist.history)
histories['epoch'] = hist.epoch
print(histories.columns)
histories_file = os.path.join(model_folder, run_name_acc + '.csv')
histories.to_csv(histories_file, index=False)

NameError: name 'hist' is not defined

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(hist.history['loss'], color='b')
plt.plot(hist.history['val_loss'], color='r')
plt.show()
plt.plot(hist.history['acc'], color='b')
plt.plot(hist.history['val_acc'], color='r')
plt.show()

In [ ]:
def saveModel(model, run_name):
    cwd = os.getcwd()
    modelPath = os.path.join(cwd, 'model')
    if not os.path.isdir(modelPath):
        os.mkdir(modelPath)
    weigthsFile = os.path.join(modelPath, run_name + '.h5')
    model.save(weigthsFile)
saveModel(model, run_name_acc)

## Predict

In [ ]:
y_pred = model.predict(x_test, batch_size=128)
print(y_pred.shape)

In [ ]:
# 这里证明os.listdir()得到的图片名称list不正确
files = os.listdir(os.path.join(cwd, 'input', 'data_test', 'test'))
print(files[:10])

In [ ]:
# 这里证明ImageDataGenerator()得到的图片名称list才是正确
gen = ImageDataGenerator()
image_size = (299, 299)
batch_size = 128
test_generator  = gen.flow_from_directory(test_folder, image_size, shuffle=False, batch_size=batch_size)
print('test_generator')
print(len(test_generator.filenames))
print(test_generator.filenames[:10])

In [ ]:
%%time
max_indexes = np.argmax(y_pred, -1)
print(max_indexes.shape)

test_dict = {}
for i, paire in enumerate(zip(test_generator.filenames, max_indexes)):
    image_name, indx = paire[0], paire[1]
    image_id = image_name[5:-4]
#     test_dict[image_id] = '%d %.4f' % (indx, y_pred[i, indx])
    test_dict[image_id] = '%d %.4f' % (indx, y_pred[i, indx])

#确认图片的id是否能与ImageDataGenerator()对应上
for key in list(test_dict.keys())[:10]:
    print('%s  %s' % (key, test_dict[key]))

In [ ]:
display(sample_submission_csv.head(2))

In [ ]:
%%time
len_sample_submission_csv = len(sample_submission_csv)
print('len(len_sample_submission_csv)=%d' % len_sample_submission_csv)
count = 0
for i in range(len_sample_submission_csv):
    image_id = sample_submission_csv.iloc[i, 0]
#     landmarks = sample_submission_csv.iloc[i, 1]
    if image_id in test_dict:
        pred_landmarks = test_dict[image_id]
#         print('%s  %s' % (image_id, pred_landmarks))
        sample_submission_csv.iloc[i, 1] = pred_landmarks
    else:
#         print(image_id)
#         sample_submission_csv.iloc[i, 1] = '9633 1.0' # 属于9633的类最多，所以全都设置成这个类，可能会比设置成空得到的结果好
        sample_submission_csv.iloc[i, 1] = '' # 设置成空
    count += 1
    if count % 10000 == 0:
        print(int(count/10000), end=' ')
display(sample_submission_csv.head(2))

In [ ]:
pred_file = os.path.join(output_folder, 'pred_' + run_name_acc + '.csv')
sample_submission_csv.to_csv(pred_file, index=None)

In [ ]:
print(run_name_acc)
print('Done !')